In [ ]:
### Construct the phonetic dictionary
eSPEDictV2 = dict()
with open('eSPEPhonologicalTableV2') as openFile:
    for line in openFile.readlines():
        line = line.strip().split('\t')
        eSPEDictV2[line[0]] = np.array([eval(x) for x in line[1:]])

In [ ]:
### Calculate energy of score (8 classes + 1 class:#(words))
def getTweetScoreV2(tweet):
    wordCount = 0
    words = tokenize(tweet)
    score = np.zeros((8,), dtype=np.int)
    for word in words:
        # 字典裡找不到這個字
        cmu, ipa = cv.convert(word)
        ipa = re.sub('[ˌˈ ]' ,'' ,ipa)
        if '*' in ipa:
            continue
        wordCount += 1
        i = 0
        while i < len(ipa):
            # 最後一個音標
            if i == len(ipa)-1:
                sym = ipa[i]
                score += eSPEDictV2[sym]
                i += 1
            # 非最後一個音標
            else:
                try:
                    sym = ipa[i] + ipa[i+1]
                    score += eSPEDictV2[sym]
                    i += 2
                except KeyError:
                    sym = ipa[i]
                    score += eSPEDictV2[sym]
                    i += 1

    score = np.append(score, wordCount)
    return score

# For Patient Classification

In [ ]:
# Patient
progress = initializeProgressBar(len(groupTextCase1))	
groupScoresV2 = []
for i, person in enumerate(groupTextCase1):	# Type of groupTextCase1: List
    progress.value += 1
    progress.description = 'Task {0}/{1}'.format(i+1, len(groupTextCase1))
    tweets = person.split('\n')
    score = np.zeros((9,), dtype=np.int)
    for tweet in tweets:
        score += getTweetScoreV2(tweet)
    groupScoresV2.append(list(score))

In [ ]:
### Ordinary People
progress = initializeProgressBar(len(baseTextCase1))
baseScoresV2 = []
for i, person in enumerate(baseTextCase1):
    progress.value += 1
    progress.description = 'Task {0}/{1}'.format(i+1, len(baseTextCase1))
    tweets = person.split('\n')
    score = np.zeros((9,), dtype=np.int)
    for tweet in tweets:
        score += getTweetScoreV2(tweet)
    baseScoresV2.append(list(score))

In [ ]:
### RandomForest Classifier
sss = StratifiedShuffleSplit(Y, 10, random_state=randint(0,65536))
classifierV2 = RandomForestClassifier(n_jobs=-1, max_features="sqrt", n_estimators=128)
X = np.array(baseScoresV2 + groupScoresV2)
Y = np.array([0] * len(baseScoresV2) + [1]*len(groupScoresV2), dtype=int)
precisionsV2 = []
for train_index, test_index in sss:
    print(train_index, test_index)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    classifierV2 = classifierV2.fit(X_train, Y_train)
    precision = classifierV2.score(X_test, Y_test)
    precisionsV2.append(precision)